In [2]:
import os
import sys; sys.path.append('..')
import torch
from torch.autograd import grad
import transformers
device = 'cuda:1'
import pandas as pd

In [3]:
def load_tokenizer_pair():
    encoder_tok = transformers.BertTokenizerFast.from_pretrained('bert-base-uncased')
    decoder_tok = transformers.BertTokenizerFast.from_pretrained('bert-base-chinese')
    return encoder_tok, decoder_tok

source_tok, target_tok = load_tokenizer_pair()

In [4]:
def word2emb(wordstr, tokenizer, embedding):
    ids = torch.tensor([tokenizer.convert_tokens_to_ids(target_tok.tokenize(wordstr))], device=device)
    emb = embedding(ids).mean(dim=1).squeeze(dim=0)
    return emb

In [5]:
# en_wordpairs = [line.strip().split(',') for line in open('difftranslation_words.txt', 'rt').readlines()]

# Same translation

In [12]:
en_wordpairs = [line.strip().split(',') for line in open('sametranslation_words.txt', 'rt').readlines()]
dirpath = '../ckp/en_zh_finetune.1000iter/'

simtable = {}
for i in range(11):
    encoder_wordemb = torch.load(f'{dirpath}/0{i:0>2}00.encoder_wordemb.pt', map_location=device)
    sim = []
    for word1, word2 in en_wordpairs:
        src0_emb = word2emb(word1, source_tok, encoder_wordemb)
        src1_emb = word2emb(word2, source_tok, encoder_wordemb)

        cos = src0_emb @ src1_emb / src0_emb.norm() / src1_emb.norm()
        sim.append(cos.item())
    simtable[i] = sim

simtable = pd.DataFrame(simtable, index=['_'.join(pair) for pair in en_wordpairs]).T
simtable.to_csv(f'{dirpath}/en_zh.sametranslation_wordsim.csv')
simtable

,ability_competence,chance_opportunity,day_date,danger_risk,force_power,goal_aim,home_family,hope_wish,mistake_error,practice_exercise,...,exist_live,estimate_evaluate,find_discover,organize_arrange,plan_design,reality_truth,receive_accept,tell_inform,anxious_worried,noisy_loud
0,0.614724,0.601071,0.332346,0.536509,0.358307,0.487317,0.361133,0.308228,0.420212,0.631950,...,0.207196,0.637766,0.333293,0.650548,0.373715,0.445783,0.449460,0.562064,0.609474,0.492827
1,0.614111,0.600372,0.332088,0.536802,0.357569,0.487416,0.360493,0.308565,0.418658,0.631506,...,0.207185,0.638205,0.331972,0.650855,0.374592,0.446642,0.449914,0.561842,0.609738,0.493081
2,0.613631,0.599871,0.329714,0.535873,0.359112,0.483895,0.361612,0.309738,0.418096,0.631651,...,0.207729,0.636340,0.332097,0.650355,0.372535,0.447200,0.452156,0.561116,0.607670,0.492843
3,0.613630,0.599870,0.329714,0.535873,0.359113,0.483895,0.361606,0.309735,0.418096,0.631650,...,0.207732,0.636338,0.332096,0.650354,0.372534,0.447200,0.452157,0.561116,0.607669,0.492844
4,0.613630,0.599870,0.329714,0.535875,0.359112,0.483895,0.361607,0.309735,0.418094,0.631650,...,0.207736,0.636339,0.332098,0.650355,0.372534,0.447200,0.452157,0.561116,0.607669,0.492844
5,0.613630,0.599870,0.329714,0.535877,0.359112,0.483895,0.361607,0.309735,0.418094,0.631648,...,0.207736,0.636339,0.332097,0.650355,0.372534,0.447200,0.452157,0.561116,0.607669,0.492844
6,0.613630,0.599870,0.329714,0.535877,0.359112,0.483894,0.361607,0.309735,0.418094,0.631648,...,0.207736,0.636339,0.332098,0.650355,0.372534,0.447200,0.452157,0.561116,0.607669,0.492844
7,0.613630,0.599870,0.329714,0.535877,0.359112,0.483894,0.361606,0.309735,0.418094,0.631648,...,0.207736,0.636339,0.332097,0.650355,0.372534,0.447200,0.452157,0.561116,0.607669,0.492844
8,0.613630,0.599870,0.329714,0.535877,0.359112,0.483894,0.361606,0.309735,0.418094,0.631648,...,0.207736,0.636339,0.332097,0.650355,0.372534,0.447200,0.452157,0.561116,0.607669,0.492844
9,0.613630,0.599870,0.329714,0.535877,0.359112,0.483894,0.361606,0.309735,0.418094,0.631648,...,0.207736,0.636339,0.332097,0.650355,0.372534,0.447200,0.452157,0.561116,0.607669,0.492844


In [13]:
en_wordpairs = [line.strip().split(',') for line in open('sametranslation_words.txt', 'rt').readlines()]
dirpath = '../ckp/zh_en_finetune.1000iter/'

simtable = {}
for i in range(11):
    decoder_wordemb = torch.load(f'{dirpath}/0{i:0>2}00.decoder_wordemb.pt', map_location=device)
    sim = []
    for word1, word2 in en_wordpairs:
        src0_emb = word2emb(word1, source_tok, decoder_wordemb)
        src1_emb = word2emb(word2, source_tok, decoder_wordemb)

        cos = src0_emb @ src1_emb / src0_emb.norm() / src1_emb.norm()
        sim.append(cos.item())
    simtable[i] = sim

simtable = pd.DataFrame(simtable, index=['_'.join(pair) for pair in en_wordpairs]).T
simtable.to_csv(f'{dirpath}/zh_en.sametranslation_wordsim.csv')
simtable

,ability_competence,chance_opportunity,day_date,danger_risk,force_power,goal_aim,home_family,hope_wish,mistake_error,practice_exercise,...,exist_live,estimate_evaluate,find_discover,organize_arrange,plan_design,reality_truth,receive_accept,tell_inform,anxious_worried,noisy_loud
0,0.614724,0.601071,0.332346,0.536509,0.358307,0.487317,0.361133,0.308228,0.420212,0.631950,...,0.207196,0.637766,0.333293,0.650548,0.373715,0.445783,0.449460,0.562064,0.609474,0.492827
1,0.612883,0.603801,0.329388,0.539991,0.354986,0.482197,0.361517,0.316763,0.424960,0.633453,...,0.218352,0.637416,0.336049,0.652615,0.367415,0.441045,0.451621,0.556272,0.603094,0.487304
2,0.610274,0.598520,0.328277,0.530261,0.347264,0.472878,0.351743,0.320820,0.427704,0.627339,...,0.211916,0.634524,0.311852,0.652478,0.363592,0.438664,0.437674,0.555511,0.595646,0.483169
3,0.612510,0.631236,0.329002,0.570900,0.333705,0.480395,0.371600,0.374979,0.485588,0.661880,...,0.220961,0.653199,0.326544,0.668714,0.367322,0.442173,0.448036,0.561033,0.612754,0.487788
4,0.601130,0.636928,0.319460,0.564225,0.334236,0.462607,0.384978,0.372945,0.490367,0.657124,...,0.213362,0.662262,0.356657,0.676246,0.374555,0.438070,0.438784,0.567181,0.617420,0.480545
5,0.614051,0.655002,0.338985,0.546277,0.342553,0.460416,0.335048,0.349572,0.500051,0.659128,...,0.223246,0.618331,0.341957,0.667802,0.349944,0.438138,0.444553,0.544738,0.598927,0.476883
6,0.613899,0.651539,0.336397,0.546616,0.330495,0.456292,0.350812,0.368205,0.511096,0.661260,...,0.228528,0.630837,0.346208,0.664938,0.348190,0.439272,0.445091,0.545125,0.598143,0.468099
7,0.615706,0.650296,0.336035,0.552635,0.337135,0.457837,0.352439,0.345920,0.499540,0.665254,...,0.230792,0.635860,0.337373,0.669787,0.343227,0.437665,0.456292,0.539720,0.603617,0.486383
8,0.615469,0.657446,0.339282,0.540291,0.334623,0.457665,0.354985,0.347797,0.486365,0.662906,...,0.229699,0.631973,0.347515,0.667395,0.346200,0.440618,0.452120,0.541420,0.604061,0.469453
9,0.615390,0.655224,0.339939,0.542110,0.334954,0.457272,0.357120,0.347856,0.485333,0.663107,...,0.229037,0.632725,0.343855,0.669011,0.351457,0.442546,0.447611,0.545456,0.606103,0.470500


# Different translation

In [14]:
en_wordpairs = [line.strip().split(',') for line in open('difftranslation_words.txt', 'rt').readlines()]
dirpath = '../ckp/en_zh_finetune.1000iter/'

simtable = {}
for i in range(11):
    encoder_wordemb = torch.load(f'{dirpath}/0{i:0>2}00.encoder_wordemb.pt', map_location=device)
    sim = []
    for word1, word2 in en_wordpairs:
        src0_emb = word2emb(word1, source_tok, encoder_wordemb)
        src1_emb = word2emb(word2, source_tok, encoder_wordemb)

        cos = src0_emb @ src1_emb / src0_emb.norm() / src1_emb.norm()
        sim.append(cos.item())
    simtable[i] = sim

simtable = pd.DataFrame(simtable, index=['_'.join(pair) for pair in en_wordpairs]).T
simtable.to_csv(f'{dirpath}/en_zh.difftranslation_wordsim.csv')
simtable

,matter_value,surface_history,society_moment,delay_century,union_space,science_range,separate_thick,shelter_technique,solid_different,favor_struggle,...,deny_hurry,cry_revenue,award_extend,aspect_variable,worship_quantity,split_reserve,survival_network,mature_logical,cast_raise,core_branc
0,0.267193,0.215696,0.293058,0.564680,0.226750,0.338206,0.630412,0.544289,0.579061,0.628189,...,0.531715,0.376378,0.342387,0.497133,0.639156,0.622702,0.347698,0.457417,0.524609,0.370613
1,0.267169,0.215311,0.293192,0.564895,0.227288,0.337199,0.630325,0.544436,0.579752,0.627807,...,0.532087,0.376369,0.342708,0.496726,0.639212,0.623063,0.348206,0.457329,0.524335,0.368884
2,0.268492,0.214790,0.291721,0.563903,0.222473,0.331959,0.630575,0.544884,0.574507,0.624611,...,0.532026,0.376325,0.340431,0.495425,0.636582,0.622954,0.349356,0.454080,0.524273,0.368888
3,0.268492,0.214790,0.291720,0.563894,0.222472,0.331958,0.630575,0.544891,0.574506,0.624609,...,0.532026,0.376325,0.340430,0.495422,0.636581,0.622953,0.349355,0.454079,0.524284,0.368888
4,0.268493,0.214788,0.291720,0.563890,0.222473,0.331958,0.630575,0.544890,0.574505,0.624609,...,0.532026,0.376324,0.340430,0.495422,0.636581,0.622952,0.349355,0.454077,0.524285,0.368887
5,0.268493,0.214788,0.291720,0.563888,0.222473,0.331958,0.630575,0.544890,0.574505,0.624609,...,0.532026,0.376324,0.340430,0.495422,0.636581,0.622952,0.349355,0.454077,0.524285,0.368887
6,0.268493,0.214787,0.291720,0.563888,0.222472,0.331958,0.630575,0.544890,0.574505,0.624609,...,0.532026,0.376324,0.340430,0.495422,0.636581,0.622952,0.349355,0.454077,0.524285,0.368887
7,0.268493,0.214786,0.291720,0.563888,0.222472,0.331959,0.630575,0.544890,0.574505,0.624609,...,0.532026,0.376324,0.340430,0.495422,0.636581,0.622952,0.349355,0.454077,0.524285,0.368887
8,0.268493,0.214786,0.291720,0.563888,0.222472,0.331958,0.630575,0.544890,0.574505,0.624609,...,0.532026,0.376325,0.340430,0.495422,0.636581,0.622952,0.349355,0.454077,0.524285,0.368886
9,0.268493,0.214786,0.291720,0.563888,0.222472,0.331958,0.630575,0.544890,0.574505,0.624609,...,0.532026,0.376325,0.340430,0.495422,0.636581,0.622952,0.349355,0.454077,0.524285,0.368886


In [15]:
en_wordpairs = [line.strip().split(',') for line in open('difftranslation_words.txt', 'rt').readlines()]
dirpath = '../ckp/zh_en_finetune.1000iter/'

simtable = {}
for i in range(11):
    decoder_wordemb = torch.load(f'{dirpath}/0{i:0>2}00.decoder_wordemb.pt', map_location=device)
    sim = []
    for word1, word2 in en_wordpairs:
        src0_emb = word2emb(word1, source_tok, decoder_wordemb)
        src1_emb = word2emb(word2, source_tok, decoder_wordemb)

        cos = src0_emb @ src1_emb / src0_emb.norm() / src1_emb.norm()
        sim.append(cos.item())
    simtable[i] = sim

simtable = pd.DataFrame(simtable, index=['_'.join(pair) for pair in en_wordpairs]).T
simtable.to_csv(f'{dirpath}/zh_en.difftranslation_wordsim.csv')
simtable

,matter_value,surface_history,society_moment,delay_century,union_space,science_range,separate_thick,shelter_technique,solid_different,favor_struggle,...,deny_hurry,cry_revenue,award_extend,aspect_variable,worship_quantity,split_reserve,survival_network,mature_logical,cast_raise,core_branc
0,0.267193,0.215696,0.293058,0.564680,0.226750,0.338206,0.630412,0.544289,0.579061,0.628189,...,0.531715,0.376378,0.342387,0.497133,0.639156,0.622702,0.347698,0.457417,0.524609,0.370613
1,0.267614,0.219385,0.293796,0.565613,0.220099,0.323374,0.626606,0.553879,0.583324,0.621591,...,0.532979,0.379617,0.351304,0.499545,0.637445,0.620640,0.339436,0.454506,0.524998,0.374503
2,0.266008,0.218976,0.283866,0.561017,0.216153,0.312139,0.624065,0.547027,0.576566,0.618378,...,0.532504,0.378705,0.358779,0.489151,0.632040,0.617268,0.333342,0.452068,0.526222,0.368521
3,0.291304,0.231541,0.280491,0.582275,0.212819,0.330393,0.622030,0.558053,0.597387,0.635243,...,0.576692,0.401828,0.387737,0.528420,0.649931,0.624761,0.349146,0.474836,0.558695,0.385604
4,0.318093,0.249136,0.293543,0.592184,0.227580,0.329154,0.635438,0.587448,0.601969,0.632482,...,0.589757,0.391167,0.404394,0.547032,0.643198,0.634507,0.369094,0.468826,0.557070,0.396854
5,0.287645,0.218921,0.299478,0.605502,0.197802,0.308007,0.631364,0.603453,0.602936,0.612941,...,0.594079,0.393303,0.410341,0.556581,0.640465,0.623599,0.319829,0.466715,0.548152,0.384910
6,0.287481,0.208629,0.294104,0.608805,0.195433,0.277796,0.628224,0.551114,0.598472,0.623766,...,0.580164,0.381722,0.399995,0.551435,0.644085,0.619934,0.333162,0.455201,0.544358,0.395498
7,0.261993,0.204779,0.301223,0.611249,0.196673,0.289548,0.627387,0.541303,0.601979,0.620674,...,0.564699,0.404708,0.419487,0.524495,0.651542,0.619130,0.317695,0.459714,0.541961,0.383305
8,0.280482,0.210208,0.291200,0.613105,0.210327,0.299781,0.626733,0.543586,0.598037,0.621440,...,0.569670,0.390768,0.391238,0.526985,0.652657,0.621590,0.329491,0.460819,0.540227,0.378783
9,0.283847,0.214359,0.292535,0.610571,0.220472,0.312151,0.626948,0.537614,0.593392,0.620637,...,0.570632,0.392563,0.382829,0.526830,0.652410,0.623977,0.333717,0.462327,0.545329,0.382542
